In [2]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# Data Imputation and Scaling for Neural Networks
Convert features so they can be used in a neural network (or GNN).  
* Impute missing features
* Scale values
* Set an "alternative" NAICS for encoding
  * For NAICS not in the training data, use the highest-volume NAICS within the sector

For continuous features, I do quantile encoding then min/max scaling to -1, 1.  For binary fields (and fields with a small number of levels) I just min/max rescale

*This script takes about 5 minutes on my MacBook Air*

In [3]:
import pandas as pd
import numpy as np

In [4]:
from pathlib import Path
import importlib, pickle
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder

In [5]:
# Imputer object for easy dataset conversion to GNN friendly format
from sba_gnn.sba_gnn import sg_imputer 
from sba_gnn.sba_gnn.sg_imputer import GNNImputer

## Input training data, set alternate NAICS

Import the data, and set an alternate NAICS for test and validation, such that NAICS that are not in the training data are set to the most common NAICS in the same sector in the training data

##### Import train data

In [6]:
sba_loans = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_transformed.parquet'))

## Get alternative NAICS (test and validation)
Map unseen NAICS using hierarchical information.  Use the highest-volume code in a grouping

In [7]:
def get_common_naics(data, group_var= 'NAICS_sector', alt_var = 'NAICS_alt'):
    """Get an alternative NAICS as the most common value in a larger group"""
    grouped = data.groupby(group_var)
    alt_naics = pd.concat([g[['NAICS']].value_counts().head(1).reset_index() \
                              for n, g in grouped], keys=[n for n, g in grouped]) \
        .reset_index(level=0) \
        .reset_index(drop = True) \
        .rename(columns={'level_0':group_var, 'NAICS':alt_var}) \
        [[group_var, alt_var]] 
    return alt_naics

In [8]:
def remap_naics(data, group_var= 'NAICS_sector', alt_var = 'NAICS_alt'):
    """Map unseen test/validation codes to same-group codes"""
    train_data = data[data['dset'] == 'train']
    oth_data = data[data['dset'] != 'train']
    
    # Do not modify the train data 
    naics_alt_train = train_data[['NAICS', 'LoanNr_ChkDgt']].rename(columns={'NAICS':alt_var})
    
    # Get alternative NAICS 
    grp_alt = get_common_naics(train_data, group_var, alt_var)
    
    # Identify seen/unseen codes
    train_naics = train_data[['NAICS']].drop_duplicates()
    naics_alt_merge = oth_data[['LoanNr_ChkDgt', 'NAICS', group_var]] \
        .merge(train_naics, how='left', on='NAICS', indicator = True)
    
    # Don't map seen codes
    naics_alt_2 = naics_alt_merge[naics_alt_merge['_merge'] == 'both'] \
        .drop(columns=['_merge', group_var], errors='ignore') \
        .rename(columns={'NAICS':alt_var})
    
    # Map unseen codes
    naics_alt_3 = naics_alt_merge[naics_alt_merge['_merge'] == 'left_only'] \
        .drop(columns=['_merge']) \
        .merge(grp_alt, how='left', on=group_var) \
        .drop(columns=[group_var])
    
    # If no match, use same unseen code
    naics_alt_3[alt_var] = naics_alt_3[alt_var].fillna(naics_alt_3['NAICS'])
    naics_alt_3.drop(columns='NAICS', inplace=True)
    
    naics_alt = pd.concat([naics_alt_train, naics_alt_2, naics_alt_3])[['LoanNr_ChkDgt', alt_var]]
    
    return data.drop(columns=[alt_var], errors='ignore') \
        .merge(naics_alt, on='LoanNr_ChkDgt')        

##### Sector map

In [9]:
sba_loans = remap_naics(sba_loans)

In [10]:
sba_loans.shape

(688081, 51)

In [11]:
(sba_loans['NAICS_alt'] == sba_loans['NAICS']).value_counts()

True     654745
False     33336
Name: count, dtype: int64

In [12]:
sba_loans['NAICS_alt'].isna().value_counts()

NAICS_alt
False    688081
Name: count, dtype: int64

##### Subsector map

In [13]:
sba_loans['naics3'] = sba_loans['NAICS'].str[0:3]

In [15]:
sba_loans = remap_naics(sba_loans, group_var='naics3', alt_var = 'NAICS_alt3')

In [16]:
(sba_loans['NAICS_alt3'] == sba_loans['NAICS']).value_counts()

True     654746
False     33335
Name: count, dtype: int64

In [17]:
sba_loans['NAICS_alt3'].isna().value_counts()

NAICS_alt3
False    688081
Name: count, dtype: int64

##### Industry group map

In [18]:
sba_loans['naics4'] = sba_loans['NAICS'].str[0:4]

In [19]:
sba_loans = remap_naics(sba_loans, group_var='naics4', alt_var = 'NAICS_alt4')

In [20]:
(sba_loans['NAICS_alt4'] == sba_loans['NAICS']).value_counts()

True     659242
False     28839
Name: count, dtype: int64

In [21]:
sba_loans['NAICS_alt4'].isna().value_counts()

NAICS_alt4
False    688081
Name: count, dtype: int64

## Imputer

In [22]:
train_df = sba_loans[sba_loans['dset'] == 'train']
test_val_df = sba_loans[sba_loans['dset'] != 'train']

For a neural network, I need to handle missings and also scale features.  For missing values, I will do a simple median fill for all, but add missing indicators.  

To scale the data, I will do a quantile transform for features with > 5 levels.  This is to avoid scaling binary features.  Then I do a Min/Max scaling on all features, so they are in the [0,1] range

I define a class to do this in sg_imputer.  This class wraps several scikit-learn imputers/scalers so that I can easily fit the objects and then transform the data.  In this section, I show some descriptives to justify my imputation/scaling choices, and also fit the scaler

In [23]:
train_df = sba_loans[sba_loans['dset'] == 'train']

##### Descriptives - explore missingness

In [24]:
# Same information, more compact
pd.concat([pd.DataFrame([train_df[c].isna().sum()]) for c in setup.predictor_features],
         keys=setup.predictor_features).set_axis(['count_missing'], axis=1)

,,count_missing
NoEmp,0,0
CreateJob,0,0
LowDoc,0,3117
DisbursementGross,0,0
new_business,0,544
urban_flag,0,83339
franchise_flag,0,0


Most fields aren't missing too much.  However I must handle those that are.  I will use a median fill, especially since many of the fields with missingness seem to be binary

##### Explore scaling

In [25]:
# Level counts
feature_level_df = pd.concat([pd.DataFrame([train_df[c].value_counts().count()]) for c in setup.predictor_features],
         keys=setup.predictor_features)
feature_level_df

,,0
NoEmp,0,425
CreateJob,0,184
LowDoc,0,2
DisbursementGross,0,78795
new_business,0,2
urban_flag,0,2
franchise_flag,0,2


A number of features are binary.  I don't need to scale these.  The others must be scaled.  

What I will do is scale features with more than a threshold number of levels using a quantile scaler.  Then I'll use a MinMax scaler on everything to be safe (as a 2 level feature could have values other than 0,1).

##### Create imputer to do the median fill and scaling
This imputer fills missing values, adding missingness indicator features.  Then it quantile scales features with over 5 levels, and MinMax scales the rest.

In [26]:
importlib.reload(sg_imputer)
from sba_gnn.sba_gnn.sg_imputer import GNNImputer

In [27]:
imputer = GNNImputer(features = setup.predictor_features,
                    naics_features = ['NAICS', 'NAICS_alt', 'NAICS_alt3', 'NAICS_alt4'])                               

In [28]:
train_out = imputer.fit_transform(train_df)

In [29]:
train_out.describe()

,NoEmp,CreateJob,LowDoc,DisbursementGross,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag,NAICS,NAICS_alt,NAICS_alt3,NAICS_alt4
count,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000
mean,-0.003650,-0.424033,-0.805820,-0.000803,-0.440904,0.702341,-0.898283,-0.985352,-0.997444,-0.608364,816.985373,816.985373,816.985373,816.985373
std,0.574311,0.795742,0.592162,0.583255,0.897555,0.711842,0.439417,0.170532,0.071459,0.793659,342.255738,342.255738,342.255738,342.255738
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000
25%,-0.440440,-1.000000,-1.000000,-0.509510,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,687.000000,687.000000,687.000000,687.000000
50%,0.022022,-1.000000,-1.000000,-0.003003,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,897.000000,897.000000,897.000000,897.000000
75%,0.499499,0.558559,-1.000000,0.514095,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1094.000000,1094.000000,1094.000000,1094.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1232.000000,1232.000000,1232.000000,1232.000000


In [30]:
with open(Path(setup.temp_path).joinpath('10_DATA_imputer.pkl'), 'wb') as fout:
    pickle.dump(imputer, fout)

In [31]:
with open(Path(setup.temp_path).joinpath('10_DATA_features.pkl'), 'wb') as fout:
    pickle.dump(imputer.features_out, fout)

In [32]:
naics_max_levels = imputer.get_naics_encoder_levels()
print(f'NAICS encoder max num: {naics_max_levels}')
with open(Path(setup.temp_path).joinpath('10_DATA_naics_max_levels.pkl'), 'wb') as fout:
    pickle.dump(naics_max_levels, fout)

NAICS encoder max num: 1232


## Transform Test, Validation Data

In [33]:
test_df = sba_loans[sba_loans['dset'] == 'test']
val_df = sba_loans[sba_loans['dset'] == 'val']

In [34]:
test_out = imputer.transform(test_df)

In [35]:
val_out = imputer.transform(val_df)

In [36]:
pd.concat([test_out[['DisbursementGross', 'NoEmp']], test_df[['DisbursementGross', 'NoEmp']]],
          axis=1).corr(method='spearman')

,DisbursementGross,NoEmp,DisbursementGross,NoEmp
DisbursementGross,1.000000,0.442105,1.000000,0.442105
NoEmp,0.442105,1.000000,0.442105,1.000000
DisbursementGross,1.000000,0.442105,1.000000,0.442105
NoEmp,0.442105,1.000000,0.442105,1.000000


In [37]:
val_out.describe()

,NoEmp,CreateJob,LowDoc,DisbursementGross,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag,NAICS,NAICS_alt,NAICS_alt3,NAICS_alt4
count,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000
mean,-0.005281,-0.420396,-0.802905,-0.001249,-0.442769,0.704160,-0.898049,-0.985402,-0.997937,-0.606557,814.446014,814.470783,814.458248,814.457399
std,0.574275,0.796882,0.596109,0.583268,0.896639,0.710044,0.439897,0.170247,0.064198,0.795043,343.605139,343.572120,343.582323,343.584014
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,1.000000,0.000000,0.000000
25%,-0.440440,-1.000000,-1.000000,-0.511081,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,680.000000,680.000000,680.000000,680.000000
50%,0.022022,-1.000000,-1.000000,-0.003003,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,890.000000,890.000000,890.000000,890.000000
75%,0.499499,0.558559,-1.000000,0.513514,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1093.000000,1093.000000,1093.000000,1093.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1232.000000,1232.000000,1232.000000,1232.000000


In [38]:
test_out.describe()

,NoEmp,CreateJob,LowDoc,DisbursementGross,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag,NAICS,NAICS_alt,NAICS_alt3,NAICS_alt4
count,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000
mean,0.010020,-0.423566,-0.802134,-0.003390,-0.455448,0.715477,-0.903889,-0.985122,-0.997420,-0.582223,616.290606,801.504654,800.886446,796.554696
std,0.576731,0.796105,0.597146,0.582376,0.890266,0.698639,0.427769,0.171855,0.071784,0.813032,460.186502,348.359537,352.009014,361.009055
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,1.000000,1.000000,0.000000
25%,-0.440440,-1.000000,-1.000000,-0.512507,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,31.250000,625.000000,669.000000,664.000000
50%,0.022022,-1.000000,-1.000000,-0.003003,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,756.000000,854.000000,855.000000,855.000000
75%,0.499499,0.558559,-1.000000,0.510119,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1057.000000,1093.000000,1093.000000,1093.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1232.000000,1232.000000,1232.000000,1232.000000


In [39]:
test_out[['NAICS', 'NAICS_alt']].corr()

,NAICS,NAICS_alt
NAICS,1.000000,0.608976
NAICS_alt,0.608976,1.000000


In [40]:
val_out[['NAICS', 'NAICS_alt']].corr()

,NAICS,NAICS_alt
NAICS,1.000000,0.999925
NAICS_alt,0.999925,1.000000


In [41]:
train_out[['NAICS', 'NAICS_alt']].corr()

,NAICS,NAICS_alt
NAICS,1.0,1.0
NAICS_alt,1.0,1.0


## Combine data
Combine transformed datasets, with key information.  Include the business ID, target status, NAICS features, as well as transformed features from above. Save this data for later use (especially the indices which indicate train/test statustrain_df

In [42]:
naics_features = ['LoanNr_ChkDgt', 'target', 'dset_naics_holdout', 'NAICS', 'NAICS_sector', 
                  'menc_NAICS', 'cenc_NAICS', 'menc_grp_NAICS', 'NAICS_alt', 'NAICS_alt3', 'NAICS_alt4'] +  \
    [c for c in train_df.columns if c.startswith('NS__')]
print(naics_features)

['LoanNr_ChkDgt', 'target', 'dset_naics_holdout', 'NAICS', 'NAICS_sector', 'menc_NAICS', 'cenc_NAICS', 'menc_grp_NAICS', 'NAICS_alt', 'NAICS_alt3', 'NAICS_alt4', 'NS___Accommodation and Food Services', 'NS___Construction', 'NS___Health Care and Social Assistance', 'NS___Manufacturing', 'NS___Other Services (except Public Administration)', 'NS___Professional, Scientific, and Technical Services', 'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn']


In [43]:
comb_naics = pd.concat([train_df[naics_features], 
                        test_df[naics_features], 
                        val_df[naics_features]], axis=0, keys=['train', 'test', 'val']) \
    .rename(columns={'NAICS':'NAICS_orig', 'NAICS_alt':'NAICS_alt_orig', 
                     'NAICS_alt3':'NAICS_alt3_orig', 'NAICS_alt4':'NAICS_alt4_orig'}) \
    .sort_index()

In [44]:
comb_df = pd.concat([comb_naics,
                     pd.concat([train_out, test_out, val_out], 
                               axis=0, keys=['train', 'test', 'val']).sort_index()],
                    axis=1) \
    .reset_index(level=0) \
    .rename(columns={'level_0':'dset'}, errors='ignore')
print(comb_df.shape)

(688081, 35)


In [45]:
print(comb_df.columns)

Index(['dset', 'LoanNr_ChkDgt', 'target', 'dset_naics_holdout', 'NAICS_orig',
       'NAICS_sector', 'menc_NAICS', 'cenc_NAICS', 'menc_grp_NAICS',
       'NAICS_alt_orig', 'NAICS_alt3_orig', 'NAICS_alt4_orig',
       'NS___Accommodation and Food Services', 'NS___Construction',
       'NS___Health Care and Social Assistance', 'NS___Manufacturing',
       'NS___Other Services (except Public Administration)',
       'NS___Professional, Scientific, and Technical Services',
       'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn',
       'NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business',
       'urban_flag', 'franchise_flag', 'missingindicator_LowDoc',
       'missingindicator_new_business', 'missingindicator_urban_flag', 'NAICS',
       'NAICS_alt', 'NAICS_alt3', 'NAICS_alt4'],
      dtype='object')


In [46]:
comb_df.to_parquet(Path(setup.temp_path).joinpath('10_DATA_combined_scaled_all.parquet'))